<a href="https://colab.research.google.com/github/opencoca/vidsum/blob/master/Working_GPT_J_6B_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-J-6B Inference 

This notebook explores how to work with the [GPT-J-6B model](https://github.com/kingoflolz/mesh-transformer-jax/#GPT-J-6B). See the link for more details about the model, including evaluation metrics and credits.

Please note it takes ~10 minutes for this notebook to fully spin up. Sometimes you will runout of ram when attempting to setup the model. If this happens factory restart the runtime and try again. If it contines to happen ...

Start this notebok by clicking `Runtime > Run all` or clicking `⌘/Ctrl+F9`. Use `⌘` on Mac and `Ctrl` on Windows and Linux. 

Keep an eye on this as it spins up. Google likes to make sure you are not a bot and will stop things if you ignore the "I'm not a robot" popup.

## Prepare it to be used

### Install Dependencies

First we download the model and install some dependencies. This step takes at least 5 minutes (possibly longer depending on server load).

!!! **Make sure you are using a TPU runtime!** !!!

In [ ]:
#@title Install Tensorflow
!pip install tensorflow==2.5.0

     |████████████████████████████████| 454.3 MB 10 kB/s 
     |████████████████████████████████| 1.2 MB 65.0 MB/s 
     |████████████████████████████████| 4.0 MB 66.2 MB/s 
     |████████████████████████████████| 462 kB 68.4 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.41.0
    Uninstalling grpcio-1.41.0:
      Successfully uninstalled grpcio-1.41.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0


In [ ]:
#@title Install GPT-J-6B
!apt install zstd

# the "slim" version contain only bf16 weights and no optimizer parameters, which minimizes bandwidth and memory
!time wget -c https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd

!time tar -I zstd -xf step_383500_slim.tar.zstd

!git clone https://github.com/kingoflolz/mesh-transformer-jax.git
!pip install -r mesh-transformer-jax/requirements.txt

# jax 0.2.12 is required due to a regression with xmap in 0.2.13
!pip install mesh-transformer-jax/ jax==0.2.12

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 278 kB of archives.
After this operation, 1,141 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 zstd amd64 1.3.3+dfsg-2ubuntu1.2 [278 kB]
Fetched 278 kB in 1s (344 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../zstd_1.3.3+dfsg-2ubuntu1.2_amd64.deb ...
Unpacking zstd (1.3.3+dfsg-2ubuntu1.2) ...
Setting up zstd (1.3.3+dfsg-2ubuntu1.2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
--2021-10-27 15:00:41--  https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd
Resolving the-eye.eu (the-eye.eu)... 162.213.130.244
Connecting to the-eye.eu (the-eye.eu)|162.213.130.244|:443... connected.
HT

Processing ./mesh-transformer-jax
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 590 kB 5.1 MB/s 
  Created wheel for mesh-transformer: filename=mesh_transformer-0.0.0-py3-none-any.whl size=26330 sha256=b2c5586d61666657fc0f82181b297b4e710571d5bd192594501492faab7ae7e6
  Stored in directory: /root/.cache/pip/wheels/56/bd/89/b1f6b2f3d6b938d0c5812ee97756a1afd32521bea293543863
  Created wheel for jax: filename=jax-0.2.12-py3-none-any.whl size=682487 sha256=365983983953ccb773bd2375fe26abaae8badac37a8fa49e66e72163f54785c7
  Stored in directory: /root/.cache/pip/wheels/f6/4d/e5/73eec5070b77f25664c67bd793d4eb97f4

## Setup Model

You may have to restart your runtime before this will work. 



In [ ]:
#@title Setup JAX for TPU use
import os
import requests 
from jax.config import config

colab_tpu_addr = os.environ['COLAB_TPU_ADDR'].split(':')[0]
url = f'http://{colab_tpu_addr}:8475/requestversion/tpu_driver0.1_dev20210607'
requests.post(url)

# The following is required to use TPU Driver as JAX's backend.
config.FLAGS.jax_xla_backend = "tpu_driver"
config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']

Sometimes the next step errors for some reason, just run it again ¯\\\_(ツ)\_/¯

In [ ]:
#@title Import libraries
import time

import jax
from jax.experimental import maps
import numpy as np
import optax
import transformers

from mesh_transformer.checkpoint import read_ckpt
from mesh_transformer.sampling import nucleaus_sample
from mesh_transformer.transformer_shard import CausalTransformer

In [ ]:
#@title Set Network Parameters 
params = {
  "layers": 28,
  "d_model": 4096,
  "n_heads": 16,
  "n_vocab": 50400,
  "norm": "layernorm",
  "pe": "rotary",
  "pe_rotary_dims": 64,

  "seq": 2048,
  "cores_per_replica": 8,
  "per_replica_batch": 1,
}

per_replica_batch = params["per_replica_batch"]
cores_per_replica = params["cores_per_replica"]
seq = params["seq"]


params["sampler"] = nucleaus_sample

# here we "remove" the optimizer parameters from the model (as we don't need them for inference)
params["optimizer"] = optax.scale(0)

mesh_shape = (jax.device_count() // cores_per_replica, cores_per_replica)
devices = np.array(jax.devices()).reshape(mesh_shape)

maps.thread_resources.env = maps.ResourceEnv(maps.Mesh(devices, ('dp', 'mp')))

tokenizer = transformers.GPT2TokenizerFast.from_pretrained('gpt2')

In [ ]:
#@title Install Tracery 
#@markdown Tracery is a gramar for spining articles, stories, and text in general. https://github.com/aparrish/pytracery
!pip install tracery

  Created wheel for tracery: filename=tracery-0.1.1-py3-none-any.whl size=7696 sha256=85c76abb320049b671e4c7260e2417cececc9ee3733160ae3b7c0191652b0f46
  Stored in directory: /root/.cache/pip/wheels/14/38/48/da02ec3e9b648c4b52ffbdae69d9a1434e5cf621435f503486
Successfully built tracery


In [ ]:
#@title Create the network
#@markdown ...loading neural weights from the downloaded files. 
#@markdown
#@markdown  > *This can take around 5 minutes.*
total_batch = per_replica_batch * jax.device_count() // cores_per_replica

network = CausalTransformer(params)

network.state = read_ckpt(network.state, "step_383500/", devices.shape[1])

network.state = network.move_xmap(network.state, np.zeros(cores_per_replica))

/usr/local/lib/python3.7/dist-packages/jax/experimental/maps.py:412: UserWarning: xmap is an experimental feature and probably has bugs!
  warn("xmap is an experimental feature and probably has bugs!")


key shape (8, 2)
in shape (1, 2048)
dp 1
mp 8
Total parameters: 6053381344
read from disk/gcs in 34.2991s


# Run Model

Finally, we are ready to infer with the model! The first sample takes around a minute due to compilation, but after that it should only take about 10 seconds per sample.

Feel free to mess with the different sampling parameters (top_p and temp), as well as the length of the generations (gen_len, causes a recompile when initially changed - recompiles will be cached).

You can also change other things like per_replica_batch in the previous cells to change how many generations are done in parallel. A larger batch has higher latency but higher throughput when measured in tokens generated/s. This is useful for doing things like best-of-n cherry picking.

*Tip for best results: Make sure your prompt does not have any trailing spaces, which tend to confuse the model due to the BPE tokenization used during training.*

In [ ]:
# allow text wrapping in generated output: https://stackoverflow.com/a/61401455
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
def infer(context, top_p=0.9, temp=1.0, gen_len=256):
    tokens = tokenizer.encode(context)

    provided_ctx = len(tokens)
    pad_amount = seq - provided_ctx

    padded_tokens = np.pad(tokens, ((pad_amount, 0),)).astype(np.uint32)
    batched_tokens = np.array([padded_tokens] * total_batch)
    length = np.ones(total_batch, dtype=np.uint32) * len(tokens)

    start = time.time()
    output = network.generate(batched_tokens, length, gen_len, {"top_p": np.ones(total_batch) * top_p, "temp": np.ones(total_batch) * temp})

    samples = []
    decoded_tokens = output[1][0]

    for o in decoded_tokens[:, :, 0]:
      samples.append(f"\033[1m{context}\033[0m{tokenizer.decode(o)}")

    #print(f"completion done in {time.time() - start:06}s")
    #this should instead be added to the object returned infered.samples and infered.completion_time
    return samples




In [27]:
#@markdown The first time that infrences are run for a given length takes imensly longer than on repeated runs. For this reason we spin up the inference engine with a few sample runs. With this out of the way we can quickly experiment with our infrence engine. 

context = """# A NEW WAY TO WORK IN THE OLD PORT OF MONTREAL AT SPACES CITÉ MULTIMÉDIA

## The Old Port of Montreal is a"""

for i in [32,64,128,256,512,1024]:
  print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
  print(f" ## Infering context with a gen_len of {i} this should take ~ {i/16} sec. when run again ##")
  print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
  print(infer(context, gen_len=i)[0])

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
 ## Infering context with a gen_len of 32 this should take ~ 2.0 sec. when run again ##
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# A NEW WAY TO WORK IN THE OLD PORT OF MONTREAL AT SPACES CITÉ MULTIMÉDIA

## The Old Port of Montreal is a major economic driver

The Old Port of Montreal, in Quebec, Canada, is undergoing major renovations to ensure it remains the premier tourist destination in the Canadian province
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
 ## Infering context with a gen_len of 64 this should take ~ 4.0 sec. when run again ##
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# A NEW WAY TO WORK IN THE OLD PORT OF MONTREAL AT SPACES CITÉ MULTIMÉDIA

## The Old Port of Montreal is a cultural heart of the city.  
Despite its centuries-old history, many 

# Example prompts

We've compiled the model. At this point it should only take about 15 seconds per sample. The first time you run inference on a new dataset takes longer than usual because of compilation to build the graph and write data into disk buffers for fast access by CPUs/GPUs.

The following parameters can be adjusted: top_p, temp, gen_len. When changing the length of generations (gen_len), recompilation is required to maintain accuracy and consistency in results.

You can also change other things like the number of generations done in parallel and how many samples are generated. In addition, you can increase or decrease the batch size for latency/throughput tradeoffs as well. This is useful when performing best-of-n cherry picking tasks where throughput needs to be maximized with minimum batch time delay on each sample generation process.

To get the best results, make sure your prompt does not have any trailing spaces. This is because BPE tokenization can be confused by them.

## Simple Inference 

### Set Sampling Parameters


In [ ]:
#@markdown ### Leave this alone unless you know what you are doing ... or are just having fun experimenting.
#@markdown 
#@markdown - - - 
#@markdown Top_p controls diversity/quality. The higher our top_p the more rich and nuanced oir inferences can be.
top_p = 0 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Temp controls randomnes. Low temps make for mechanical inferences.
temp = 0.6 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Gen_len adjects the length of text infered.
#@markdown Specifcily it adustes the nember of tokens (~4 per engish word).
#@markdown
#@markdown *Note: Changing gen_len will cause long wait times when first setting lengs that have not been set before.*
#@markdown  
gen_len = 512 #@param {type: "slider", min:64, max: 1024, step: 64}
#@markdown - - - 
#@markdown Generate controls the number of inferences that will be generated.
generate = 1 #@param {type: "slider", min:1, max: 12, step: 1}


### Text examples

In [28]:
#@title Raw Context 
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
gen_len = 256 #@param {type: "slider", min:64, max: 512, step: 64}


#print("Enter/Paste your content. Ctrl-D or Ctrl-Z ( windows ) to save it.")
#context = []
#while True:
#    try:
#        line = input()
#    except EOFError:
#        break
#    context.append(line)

context = """"

Title: Bitcoin back above $50,000 as recovery continues
Date: 23 August
Article:

The price of Bitcoin has risen above $50,000 
(£36,480) for the first time in three months
as the cryptocurrency continues to recover 
from a deep slump.

The coin fell sharply in May after a crackdown
in China and a decision by Elon Musk's Tesla 
not to accept it as payment any more.

But investor sentiment is improving as more 
mainstream financial companies begin using
the digital currency.

Summery:"""

print(infer(top_p=top_p, temp=temp, gen_len=gen_len, context=context)[0])

"

Title: Bitcoin back above $50,000 as recovery continues
Date: 23 August
Article:

The price of Bitcoin has risen above $50,000 
(£36,480) for the first time in three months
as the cryptocurrency continues to recover 
from a deep slump.

The coin fell sharply in May after a crackdown
in China and a decision by Elon Musk's Tesla 
not to accept it as payment any more.

But investor sentiment is improving as more 
mainstream financial companies begin using
the digital currency.

Summery:

Bitcoin is over $50,000. However, we also see the
banking industry and fiat money system seems
to be stuck in a downward spiral. If Bitcoin
tanks to the $40,000 level, it is not out of
the question to see a new low.

Moreover, I believe that the next correction
is likely to start in early September. My
guess is that this new low will go back to
around $5,000.

Given the rising BTC volume, there are some
very smart people that are betting on a big
correction. Also, several well known miners
have come fo

In [ ]:
#@title Expand into a script 
#@markdown The will expand a lead into a script bassed on the opening of pulp fiction.
expand_this = "Feline biology is amazing" #@param {type:"string"}

#@markdown - - - - - - - - - - - - - - - - - - - - - - - - -
#@markdown These options don't need to be adjusted.
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
gen_len = 512 #@param {type: "slider", min:64, max: 2048, step: 64}
generate = 1 #@param {type: "slider", min:1, max: 12, step: 1}
#@markdown - - - - - - - - - - - - - - - - - - - - - - - - -


context = """

INT. COMPUTER LAB – MORNING

A normal computer lab, an office or university, Toronto Canada.
It's about 9:00 in the morning. While the place isn't jammed,
there's a healthy number of people working, drinking coffee, 
reading at their computers.

Two of these people are a YOUNG MAN and a YOUNG WOMAN. The
Young Man has a slight working-class English accent.

             YOUNG MAN
  I want to tell you about what we've been studying.

             YOUNG WOMAN
  Oh? What have you been focusing on? 

             YOUNG MAN
  {expand_this}. For example"""


for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=context.format(expand_this = expand_this))[0])
  except:
    pass



INT. COMPUTER LAB – MORNING

A normal computer lab, an office or university, Toronto Canada.
It's about 9:00 in the morning. While the place isn't jammed,
there's a healthy number of people working, drinking coffee, 
reading at their computers.

Two of these people are a YOUNG MAN and a YOUNG WOMAN. The
Young Man has a slight working-class English accent.

             YOUNG MAN
  I want to tell you about what we've been studying.

             YOUNG WOMAN
  Oh? What have you been focusing on? 

             YOUNG MAN
  Feline biology is amazing. For example, it's amazing how one
  small gene can make the difference between being a lion
  or a zebra.

             YOUNG WOMAN
  Wow, that's really interesting. Are you a student at
  university?

             YOUNG MAN
  No. I work for the Canadian Wildlife Service. 

             YOUNG WOMAN
  Wow, really? What are you doing here?

             YOUNG MAN
  I take care of the population of the entire country.

             YOUNG WOMAN


In [29]:
#@title Make a title sound more academic 
#@markdown  
improve_this = "we're going ti make an instrument using the potentiometer " #@param {type:"string"}
#@markdown - - -
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
gen_len = 64 #@param {type: "slider", min:64, max: 2048, step: 64}
generate = 12 #@param {type: "slider", min:1, max: 12, step: 1}

context = """
Instead of saying, "{improve_this}" a better shibboleth would be, \""""

#Indie Hackers want to click on titles such as 
#use something with more punch

for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=context.format(improve_this = improve_this))[0].split('"')[3])
  except:
    pass

let's make an instrument using the potentiometer 
we're going to make an instrument that uses the potentiometer.
we're going ti make an instrument to measure potential difference.
 we are going to make an instrument that uses a potentiometer, a part of a potentiometer, or a power source.
we're going to make an instrument using a potentiometer.
we're going to make a circuit using the potentiometer and a resistor.... 
we're going to use a potentiometer to make an instrument.
oh, we're going to make an instrument that uses the potentiometer
we are going to make an instrument with a potentiometer.
we're going to make a sensor that senses the state of a potentiometer.
we're going ti analyze the data, identify patterns, and draw conclusions
we're going to use the potentiometer


In [30]:
#@title Give headlines some more Clickbait 
improve_this = "Make an instrument using the a light sensor " #@param {type:"string"}
#@markdown - - - - - - - - - - - - - - - - - - - - - - - - 
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 1 #@param {type:"slider", min:0, max:1, step:0.1}
gen_len = 64 #@param {type: "slider", min:64, max: 2048, step: 64}
generate = 12 #@param {type: "slider", min:1, max: 12, step: 1}

context = """
You need to make your title more clickbate. So instead of,
"{improve_this}" use something with more punch such as,\""""


for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=context.format(improve_this = improve_this))[0].split('"')[3])
                #context=context.format(improve_this = improve_this))[0])

  except:
    pass

 Make your own light sensor. Or the absolutely most clickbate you can do is, Make your own hoover.

#3 is the best one! Smartphone creator, HP?
In April 2015, it was announced that HP, the world's biggest PC maker, would use Google's Project T

Make an instrument using the light sensor on an iPhone 
 Find your way out of your room!

just a thought. There is a place for clickbate, just don't go overboard.

Just make it informative for the person in the first place. To help them understand how to make it. I can't really find anything i disagree with, but i
 
<|endoftext|>Reformed faith churches in Lithuania

The history of Reformed Churches in Lithuania extends back to the 19th century.

History

In 1818, the Prussian army entered Lithuania, putting it under their control, including its Protestant population. By 1823, the Prussians held large areas in
 
 
 
Make an instrument using the (Stuff that is more clickbate) light sensor
 Make a lamp controlled light effect from an Arduino, or a 

In [32]:
#@title Generating inference prompts using tracery
import tracery
from tracery.modifiers import base_english

rules = {
    
    'origin': 'Our team builds #imagine# new online #location# beyond',
    'imagine': ['amazing', 'spectaculare' , 'bold', ],
    'location': ['worlds', 'spaces', 'colabortion']
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)

for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=grammar.flatten("#origin#"))[0] + "\n")
  except:
    pass

Our team builds spectaculare new online colabortion beyond the account of the ugly styles, genetic to existing cells to generate a client too diagnosed as sequence will serve 15 absorption terms per claim. They often view and write sources and are an client and content brain in able &ldquo. expanding the download Listen for leaders after feeling occurred and saving their diverse likelihood healthcare will understand

Our team builds spectaculare new online spaces beyond the desktop for the first generation to have always-on connectivity.

M/V BUSINESSMAN BRASIL - Interactive Multi-Use Interface. A revolutionary mobile device that gives the first generation an entire new platform to connect, express, share and grow their community.

Krakwad Software engineers have

Our team builds spectaculare new online colabortion beyond hormonally generic ditropan 5mg prescription.

Cvs. drenkal has decided to 5mg ditropan prescription bring with it a many drug that our endocrine us has been waiting 

In [ ]:
#@title Generating inference prompts using tracery
import tracery
from tracery.modifiers import base_english

rules = {
    
    'origin': '#this_semester# Jane #does# #good_bad# in class. We hope that she',
    'this_semester': ['Our focus this semester was learning to program microcomputers, using sensors, and building electronic circuits.','This semester we focused on learning to build electric circuits, reading information from sensors, and programming microcontrollers.'],
    'does': ['works', 'does' , 'particpates', 'performs' ],
    'good_bad': ['well', 'competently', 'baddly']
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)

for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=grammar.flatten("#origin#"))[0] + "\n")
  except:
    pass

This semester we focused on learning to build electric circuits, reading information from sensors, and programming microcontrollers. Jane performs competently in class. We hope that she continues to improve her oral and written communication skills and knowledge of the concepts that she is being taught. Jane is a person who thrives in group situations, and excels at identifying the strengths and weaknesses of others, which makes her an ideal choice to serve as a captain and in any leadership roles that she is given in

Our focus this semester was learning to program microcomputers, using sensors, and building electronic circuits. Jane does competently in class. We hope that she can keep it up through the winter and spring quarter, too. She has learned to program with BASIC, has gained a basic understanding of C, and has a good handle on digital logic.

The excellent education afforded by our schools has led us to the middle class, so it seems that none of our children

This semester we

In [ ]:
#@title Article Generator 
title = "Being your own Boss" #@param {type:"string"}
site = "" #@param {type:"string"}
date = "" #@param {type:"string"}
author = "Jim Rohn" #@param {type:"string"}
tags = "goals, drive, gtd" #@param {type:"string"}
lead = "## Table of Content" #@param {type:"string"}
#@markdown - - -

context = """
# {title}
### Published:{date}
### By:{author}
##### Tags: {tags}
### Site: {site}

{lead}
"""

for x in range(generate):
  print(infer(top_p=top_p, 
              temp=temp, 
              gen_len=gen_len, 
              context=context.format(title =title, 
                                     site = site, 
                                     date = date, 
                                     author = author, 
                                     tags = tags, 
                                     lead = lead))[0])


# Being your own Boss
### Published:
### By:Jim Rohn
##### Tags: goals, drive, gtd
### Site: 

## Table of Content

1. [Introduction](#intro)
2. [Guiding Principles](#guiding-principles)
3. [Know yourself](#know-yourself)
4. [Starting at the top](#starting-at-the-top)
5. [The two elements

# Being your own Boss
### Published:
### By:Jim Rohn
##### Tags: goals, drive, gtd
### Site: 

## Table of Content

### Welcome to _Being Your Own Boss_

### Chapter 1: Why Are You Doing This?

### Introduction

When I started doing this project, I was working for someone else, which may not seem like a major difference. They made the decisions, I had to do what I was told

# Being your own Boss
### Published:
### By:Jim Rohn
##### Tags: goals, drive, gtd
### Site: 

## Table of Content

### Introduction

### Building Motivation
### Setting Goals
### Benefits of Setting Goals
### Why Goals are Essential to Your Success
### Ways to Motivate Yourself
### Important steps to begin

### CHAPTER 1: SETTIN

In [ ]:
#@title Article generation { form-width: "300px" }
title = "COVID-19: What do vaccine passports in Quebec & Ontario mean for Canada's largest provinces? " #@param {type:"string"}
site = "https://www.canada.ca/en/public-health/services/diseases/" #@param {type:"string"}
date = "Sep 7, 2021" #@param {type:"string"}
author = "" #@param {type:"string"}
tags = "COVID-19, Quebec, Ontario, Canada" #@param {type:"string"}
lead = "The passports are certificates that confirm vaccinations and allow people to enjoy eating" #@param {type:"string"}


#print("Enter/Paste your content. Ctrl-D or Ctrl-Z ( windows ) to save it.")
#context = []
#while True:
#    try:
#        line = input()
#    except EOFError:
#        break
#    context.append(line)

context = """
# {title}
### Published:{date}
### By:{author}
##### Tags: {tags}
### Site: {site}

{lead}"""

for x in range(generate):
  print(infer(top_p=top_p, 
              temp=temp, 
              gen_len=gen_len, 
              context=context.format(title =title, 
                                     site = site, 
                                     date = date, 
                                     author = author, 
                                     tags = tags, 
                                     lead = lead))[0])


# COVID-19: What do vaccine passports in Quebec & Ontario mean for Canada's largest provinces? 
### Published:Sep 7, 2021
### By:
##### Tags: COVID-19, Quebec, Ontario, Canada
### Site: https://www.canada.ca/en/public-health/services/diseases/

The passports are certificates that confirm vaccinations and allow people to enjoy eating certain foods for a year without paying a fine. 

The COVID-19 situation in Canada is still very early. Everyone is a lot safer than they were on July 5th, 2020. This should reassure people that only a few Canadians have been infected with the virus. 

- - - -

# COVID-19: What do vaccine passports in Quebec & Ontario mean for Canada's largest provinces? 
### Published:Sep 7, 2021
### By:
##### Tags: COVID-19, Quebec, Ontario, Canada
### Site: https://www.canada.ca/en/public-health/services/diseases/

The passports are certificates that confirm vaccinations and allow people to enjoy eating healthy, including for children who are under five years old. For e

## Code Generation

### Set Sampling Parameters


In [ ]:
#@markdown ### Leave this alone unless you know what you are doing ... or are just having fun experimenting.
#@markdown 
#@markdown - - - 
#@markdown Top_p controls diversity/quality. The higher our top_p the more rich and nuanced oir inferences can be.
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Temp controls randomnes. Low temps make for mechanical inferences.
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Gen_len adjects the length of text infered.
#@markdown Specifcily it adustes the nember of tokens (~4 per engish word).
#@markdown
#@markdown *Note: Changing gen_len will cause long wait times when first setting lengs that have not been set before.*
#@markdown  
gen_len = 512 #@param {type: "slider", min:64, max: 1024, step: 64}
#@markdown - - - 
#@markdown Generate controls the number of inferences that will be generated.
generate = 1 #@param {type: "slider", min:1, max: 12, step: 1}


### Python Code

In [ ]:
#@title Python Code - Simple 
#@markdown Generate code by just writng a comment and a name for a fuction 
python_comment = "check if a number is prime or not" #@param {type:"string"}
#variable_n_val = "youtube_video_url" #@param {type:"string"}
function_name = "p" #@param {type:"string"}


context = """# {python_comment}

def is_""".format(python_comment = python_comment, 
           function_name = function_name)

for x in range(generate):
  print(infer(top_p=top_p, temp=temp, gen_len=gen_len, context=context)[0])

# check if a number is prime or not

def is_prime(n):
    i = 2
    while i * i <= n:
        if n % i == 0:
            return False
        i += 1
    return True

# create and input a list of numbers
numbers = []
for i in range(5):
    numbers.append(int(input()))

# input a value and find whether it is in the list or not
# you can use range(len(numbers)) here
def check_input(n):
    n = int(input())
    for i in range(0, len(numbers)):
        if numbers[i] == n:
            return True
        else:
            return False

# main function
def main():
    # input a value
    value = int(input())

    # checks whether the value is in the list or not
    if check_input(value):
        print('yes')
    else:
        print('no')

    # find whether the input value is prime or not
    if is_prime(value):
        print('prime')
    else:
        print('non-prime')

    # clear the input after entering values
    input()

main()

A:

You are calling the check_input() function, but it is

In [ ]:
#@title Python Code - Complex
python_comment = "detect security flaws " #@param {type:"string"}
variable_name ="person_real_name" #@param {type:"string"}
variable_val = "Isabelle Plante" #@param {type:"string"}
function_name = "online_account_scan" #@param {type:"string"}
#@markdown - - -


context = '''#!/usr/bin/env python3

# {python_comment}

{variable_name} = "{variable_val}"

def {function_name}(n):
  """This'''.format(python_comment = python_comment,
           variable_name = variable_name,
           variable_val = variable_val,
           function_name = function_name)

for x in range(generate):
  print(infer(top_p=top_p, temp=temp, gen_len=gen_len, context=context)[0])

#!/usr/bin/env python3

# detect security flaws 

person_real_name = "Isabelle Plante"

def online_account_scan(n):
  """This method checks online accounts such as Facebook, Twitter, LinkedIn, etc. and compares their
     last login time and email to a known list. In this case the list of email/accounts is a subset
     of known accounts"""
  
  n=n.replace('lastlogin,',"\n")
  n=n.replace('lastloginfrom,',"\n")
  n=n.replace('lastloginfromdomain,',"\n")
  n=n.replace('lastloginfromdomain.',","\n")
  n=n.replace('lastloginfromdomain.com',","\n")
  n=n.replace('lastloginfromdomain.co.uk',","\n")
  n=n.replace('lastloginfromdomain.com',","\n")
  n=n.replace('lastloginfromdomain.ddns.net',","\n")
  n=n.replace('lastloginfromdomain.info',","\n")
  n=n.replace('lastloginfromdomain.net',","\n")
  n=n.replace('lastloginfromdomain.org',","\n")
  n=n.replace('lastloginfromdomain.ru',","\n")
  n=n.replace('lastloginfromdomain.xx',","\n")
  n=n.replace('lastloginfromdomain.com',","\n")
  n=n.repl

In [ ]:
#@title Vue Code - Simple { form-width: "500px" }
comment = "guest speakers" #@param {type:"string"}
#variable_n_val = "youtube_video_url" #@param {type:"string"}
name = "tab_1" #@param {type:"string"}
#@markdown - - - 
#@markdown Leave this alone unless you know ...
top_p = 0.8 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
gen_len = 512 #@param {type:"slider", min:64, max:1024, step:64}
generate = 1 #@param {type: "slider", min:1, max: 12, step: 1}

context = """
<!-- Vue template for {comment} page -->
<section>
  <div id="app" class="container">
    <div class="content">
      <div id="{name} " class=""".format(comment = comment, 
           name = name)
      
print(infer(top_p=top_p, temp=temp, gen_len=gen_len, context=context)[0])


<!-- Vue template for guest speakers page -->
<section>
  <div id="app" class="container">
    <div class="content">
      <div id="tab_1 " class= "container">
        <div class="container">
          <div class="col-sm-8 col-md-8 col-xs-12">
            <div class="card-panel">
              <h3 class="mb-0 text-center">Best Companies to Work For</h3>
              <ul class="list-group list-group-flush">
                <li class="list-group-item d-flex align-items-center">
                  <div class="d-flex align-items-center">
                    <img src="https://picsum.photos/200" class="avatar">
                    <h6 class="mb-0 text-right">Veronica</h6>
                  </div>
                  <div class="d-flex align-items-center">
                    <span class="text-muted">Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.</span>
                    <small class="text-muted">Donec sed odio dui

# Examples of GPT genereated code.

The following are examples of code generated in prior runs.

Please paste interesting examples below

In [ ]:
# check if a number is prime or not

def is_prime(n):
  """ 
  > returns true if n is a prime,
  > false otherwise.
  """
  for i in range(2, n):
    if n % i == 0:
      return False
  return True

print(is_prime(7))

True


In [ ]:
# Program to check if a number is prime or not

num = 9

# Program to check if a number is prime or not

def is_prime(n):
    while n > 1:
        while n % 2 == 0:
            return False
        n = n / 2
    return True

print("This is an example of a function that works but is quite wrong!")

for i in range(2, 20):
  print(f"Is {i} prime? {is_prime(i)}")



This is an example of a function that works but is quite wrong!
Is 2 prime? False
Is 3 prime? True
Is 4 prime? False
Is 5 prime? True
Is 6 prime? False
Is 7 prime? True
Is 8 prime? False
Is 9 prime? True
Is 10 prime? False
Is 11 prime? True
Is 12 prime? False
Is 13 prime? True
Is 14 prime? False
Is 15 prime? True
Is 16 prime? False
Is 17 prime? True
Is 18 prime? False
Is 19 prime? True


In [ ]:
# check if a number is prime or not
def is_prime(n):
    for i in range(2, n):
        if n % i == 0:
            return False
    return True

is_prime(9)

False